In [80]:
import sys
sys.path.append('..')
from data.data_reader import *

In [81]:
import tqdm
import scanpy as sc
import pandas as pd

In [82]:
download_file('https://plus.figshare.com/ndownloader/files/35775512','35775512.h5ad')
adata_orig = sc.read_h5ad("35775512.h5ad")
adata_orig.X[adata_orig.X == float("inf")]=0

File downloaded successfully to 35775512.h5ad


### NOW WE DO PCA

In [83]:
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px
n_components=6
pca = PCA(n_components=n_components)
pca.fit(adata_orig.X)

new_X=pca.transform(adata_orig.X)
df_to_be_shown=pd.DataFrame(new_X,columns=[f'f{i}' for i in range(n_components)])
df_to_be_shown['control']=list(adata_orig.obs.core_control)

In [84]:
adata_orig.obs['gene_name']=list(pd.Series(adata_orig.obs.index).apply(lambda x:x.split("_")[1]))
adata_orig.obs['id']=range(adata_orig.obs.shape[0])

In [85]:
def cosine_similarity(A):
  AAt=np.matmul(A,A.transpose())
  n_A=np.sqrt((A**2).sum(axis=1)).reshape(-1,1)
  n_A=np.matmul(n_A,n_A.transpose())
  return AAt/(n_A)

In [86]:
cos_sim_f=cosine_similarity(np.array(df_to_be_shown.drop(['control'], axis=1)))

In [98]:
similarity_matrix=np.zeros(cos_sim_f.shape)
similarity_db=hu_data_loader()

for gene_name in tqdm.tqdm(adata_orig.obs.gene_name.unique()):
    query=query_hu_data(similarity_db,gene_name)
    for q in query:
        if q in adata_orig.obs.gene_name.values:
            y_indices=adata_orig.obs[adata_orig.obs.gene_name==q].id
            x_indices=adata_orig.obs[adata_orig.obs.gene_name==gene_name].id
            for x_id in x_indices:
                for y_id in y_indices:
                    similarity_matrix[y_id,x_id]=1
                    similarity_matrix[x_id,y_id]=1

cos_sim_f_flatten=cos_sim_f.reshape(-1,)
similarity_matrix_flatten=similarity_matrix.reshape(-1,)
cos_sim_f_flatten=cos_sim_f_flatten[similarity_matrix_flatten==1]

File downloaded successfully to humap2_complexes_20200809.txt


100%|██████████| 2394/2394 [00:33<00:00, 70.86it/s] 


In [99]:
choice=np.random.choice(cos_sim_f_flatten.shape[0], 2048)
cos_sim_f_flatten=cos_sim_f_flatten[choice]
cos_sim_f_flatten=pd.DataFrame(cos_sim_f_flatten,columns=['correlations'])
fig=px.violin(cos_sim_f_flatten, y='correlations',width=500, height=400)
fig.show()